In [2]:
from selenium import webdriver

In [30]:
EXE_PATH = r'C:/Users/raisi/Downloads/Chrome Driver/chromedriver'
driver = webdriver.Chrome(executable_path=EXE_PATH)
#driver.get('https://www.datasciencecentral.com/profiles/blog/list?promoted=1')

In [51]:
##scrapping 100 pages and getting the topics from each page
tpclist = []

for i in range(1,100):
    link1 = 'https://www.datasciencecentral.com/profiles/blog/list?promoted=1&page={}'.format(i)
    driver.get(link1)
    ##inside each page taking out 20 such blogs 
    for j in range(1,20):
        link2 = '//*[@id="xg_canvas"]/div[2]/div[2]/div[{}]/h3/a'.format(j)
        clslist = driver.find_elements_by_xpath(link2)
        tpc = clslist[0].get_attribute('text')
        tpclist.append(tpc)

In [55]:
## Saving it as a datafile to be used later 
import pandas as pd
tpcdf = pd.DataFrame(tpclist)


In [110]:
tpcdf.columns = ['topics']
tpcdf[:5]

,topics
0,Will AI Force Humans to Become More Human?
1,Neural Quantum States
2,Business Intelligence vs Business Analytics
3,Six AI Strategies – But Only One Winner
4,"Weekly Digest, January 6"


## Data preprocessing 

Following steps are followed: 

- Tokenization 
- Removal of stop words 
- Lemmmitization 
- Creation of bag of words 



In [61]:
## Lemmatization and removal of stop words using Spacy
import spacy
nlp = spacy.load('en_core_web_sm')

In [151]:
def preprocess1(x):
    ##removing extra white spaces
    x = ' '.join(x.split())
    ##converting the text to spacy vector
    spcyvec = nlp(x)
    prcdtxt = ''
    prcdtxtlist = []
    ##lemmatizing and removing stop words 
    for i in range(len(spcyvec)):
        d = spcyvec[i].lemma_
        if(spcyvec[i].is_stop == False):
            prcdtxtlist.append(spcyvec[i].text)
    prcdtxt = ' '.join(prcdtxtlist)
    return(prcdtxt)
     

In [152]:
tpcdf['proctext'] = tpcdf['topics'].map(preprocess1)
tpcdf[:5]

,topics,proctext
0,Will AI Force Humans to Become More Human?,AI Force Humans Human ?
1,Neural Quantum States,Neural Quantum States
2,Business Intelligence vs Business Analytics,Business Intelligence vs Business Analytics
3,Six AI Strategies – But Only One Winner,AI Strategies – Winner
4,"Weekly Digest, January 6","Weekly Digest , January 6"


In [166]:
##Making the bag of words
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
bow_processed = bow.fit_transform(tpcdf['proctext'])


In [171]:
bow_processed.toarray().shape

(1881, 2842)

In [174]:
len(bow.get_feature_names())

2842

In [176]:
##Fitting the LDA
from sklearn.decomposition import LatentDirichletAllocation as LDA

In [177]:
##fixing on number of topics = 6. The number of documents is already given , as each row of count vectorizer is a differnt document
ntopic = 6
lda = LDA(n_components = 6)
lda.fit(bow_processed)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=6, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [214]:
##Getting the topics and the top 10 words associated with it 
for index, topic in enumerate(lda.components_):
    bowf = bow.get_feature_names()
    tpcstr = 'Topic {} :'.format(index)
    print(tpcstr)
    print(" ".join([bowf[i]
                    for i in topic.argsort()[:-11:-1]])) #argument sort in order of magnitude (by default quicksort)

Topic 0 :
data ai scientist new science analytics book business big strategy
Topic 1 :
data science use cases 10 ai business big 2019 apache
Topic 2 :
data digest weekly science analysis learning time vs central series
Topic 3 :
data simple explained ai python scientists concepts science statistical english
Topic 4 :
neural networks ai regression book network analytics python free decision
Topic 5 :
learning machine data intelligence deep artificial ai science big applications


In [ ]:
from pyLDAvis import sklearn as sklearn_lda
import pyLDAvis

In [247]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = sklearn_lda.prepare(lda, bow_processed, bow)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.067009  0.188329       1        1  22.083092
2     -0.148566 -0.066880       2        1  19.618639
1     -0.018429 -0.038229       3        1  15.529309
3      0.018679 -0.086888       4        1  14.724530
4      0.184540  0.022694       5        1  14.085413
0      0.030784 -0.019026       6        1  13.959018, topic_info=     Category        Freq          Term       Total  loglift  logprob
1488  Default  219.000000      learning  219.000000  30.0000  30.0000
1563  Default  137.000000       machine  137.000000  29.0000  29.0000
776   Default   84.000000        digest   84.000000  28.0000  28.0000
2777  Default   75.000000        weekly   75.000000  27.0000  27.0000
668   Default  439.000000          data  439.000000  26.0000  26.0000
1360  Default   53.000000  intelligence   53.000000  25.0000  25.0000
2697  Default   36.000000           use   36.000000  24.0000  24.0000
1747  Default   34.000000        neural   34.000000  23.0000  23.0000
712   Default   56.000000          deep   56.000000  22.0000  22.0000
201   Default   47.000000    artificial   47.000000  21.0000  21.0000
2250  Default  175.000000       science  175.000000  20.0000  20.0000
1746  Default   25.000000      networks   25.000000  19.0000  19.0000
2339  Default   25.000000        simple   25.000000  18.0000  18.0000
395   Default   25.000000         cases   25.000000  17.0000  17.0000
2252  Default   37.000000     scientist   37.000000  16.0000  16.0000
0     Default   25.000000            10   25.000000  15.0000  15.0000
140   Default   49.000000      analysis   49.000000  14.0000  14.0000
973   Default   26.000000     explained   26.000000  13.0000  13.0000
2126  Default   29.000000    regression   29.000000  12.0000  12.0000
336   Default   38.000000          book   38.000000  11.0000  11.0000
2584  Default   28.000000          time   28.000000  10.0000  10.0000
534   Default   16.000000      concepts   16.000000   9.0000   9.0000
1752  Default   50.000000           new   50.000000   8.0000   8.0000
2253  Default   31.000000    scientists   31.000000   7.0000   7.0000
1745  Default   13.000000       network   13.000000   6.0000   6.0000
2295  Default   17.000000        series   17.000000   5.0000   5.0000
897   Default   13.000000       english   13.000000   4.0000   4.0000
307   Default   59.000000           big   59.000000   3.0000   3.0000
414   Default   19.000000       central   19.000000   2.0000   2.0000
19    Default   17.000000          2019   17.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
1496   Topic6    3.193802       lessons    4.649561   1.5935  -6.1001
2370   Topic6    3.960264      solution    6.162268   1.5269  -5.8850
1628   Topic6    3.062518         means    4.655879   1.5502  -6.1421
2271   Topic6    3.907343      security    6.169689   1.5123  -5.8984
427    Topic6    4.732259      changing    7.744816   1.4764  -5.7069
1752   Topic6   23.546825           new   50.585847   1.2044  -4.1023
2450   Topic6    8.492844      strategy   15.450235   1.3706  -5.1221
1593   Topic6    7.515446     marketing   13.850995   1.3576  -5.2443
1299   Topic6    7.732786        impact   14.705200   1.3263  -5.2158
668    Topic6   86.910129          data  439.705274   0.3478  -2.7964
1738   Topic6    6.156527          need   11.683694   1.3284  -5.4438
336    Topic6   14.074647          book   38.859947   0.9535  -4.6169
94     Topic6    5.471677      advanced   10.864429   1.2831  -5.5617
2369   Topic6    5.134058      software   10.072105   1.2952  -5.6254
114    Topic6   24.633557            ai  136.111884   0.2597  -4.0572
145    Topic6   14.741906     analytics   68.675395   0.4303  -4.5706
1069   Topic6    5.012090   forecasting   10.850944   1.1966  -5.6494
1588   Topic6    3.960223          maps    6.937079   1.4085  -5.8850
370    Topic6   10.306751

In [290]:
##Doing topic modelling using TF-IDF instead of topic modelling
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

pipe = Pipeline([('count', CountVectorizer()),
                 ('tfid', TfidfTransformer())])

pipefit = pipe.fit(tpcdf['proctext'])

In [291]:
pipefit['tfid'].idf_.shape

(2842,)

In [292]:
pipefit.transform(tpcdf['proctext']).shape

(1881, 2842)

In [294]:
tfidfprocmatrix = pipefit.transform(tpcdf['proctext'])
tfidfprocarray = tfidfprocmatrix.toarray()
tfidf_features = pipefit['count'].get_feature_names()

In [295]:
##fixing on number of topics = 6. The number of documents is already given , as each row of count vectorizer is a differnt document
ntopic = 6
lda = LDA(n_components = 6)
lda.fit(tfidfprocarray)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=6, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [299]:
##Getting the topics and the top 10 words associated with it 
for index, topic in enumerate(lda.components_):
    tfidf_features = pipefit['count'].get_feature_names()
    tpcstr = 'Topic {} :'.format(index)
    print(tpcstr)
    print(" ".join([tfidf_features[i]
                    for i in topic.argsort()[:-11:-1]])) #argument sort in order of magnitude (by default quicksort)

Topic 0 :
data analytics science ai business analysis time strategy march picture
Topic 1 :
digest weekly explained december simple english concepts statistical october picture
Topic 2 :
data digest weekly learning science spark april july september january
Topic 3 :
artificial intelligence data neural learning machine scientists networks science network
Topic 4 :
data machine learning science ai scientist digital transformation bayesian statistics
Topic 5 :
learning data use science cases machine deep ai new free


In [303]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = sklearn_lda.prepare(lda, tfidfprocmatrix, pipe['count'])
LDAvis_prepared

C:\Users\raisi\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.044898  0.065141       1        1  18.096399
3      0.073162  0.025912       2        1  17.432135
0      0.020122 -0.075111       3        1  17.084985
4      0.018805 -0.057908       4        1  16.815930
2     -0.050634  0.040258       5        1  15.811564
1     -0.106353  0.001708       6        1  14.758988, topic_info=     Category       Freq            Term      Total  loglift  logprob
2777  Default  23.000000          weekly  23.000000  30.0000  30.0000
776   Default  24.000000          digest  24.000000  29.0000  29.0000
201   Default  12.000000      artificial  12.000000  28.0000  28.0000
1360  Default  13.000000    intelligence  13.000000  27.0000  27.0000
2697  Default  10.000000             use  10.000000  26.0000  26.0000
395   Default   8.000000           cases   8.000000  25.0000  25.0000
1747  Default   9.000000          neural   9.000000  24.0000  24.0000
973   Default   7.000000       explained   7.000000  23.0000  23.0000
2611  Default   5.000000  transformation   5.000000  22.0000  22.0000
699   Default   4.000000        december   4.000000  21.0000  21.0000
276   Default   6.000000        bayesian   6.000000  20.0000  20.0000
2431  Default   4.000000      statistics   4.000000  19.0000  19.0000
897   Default   4.000000         english   4.000000  18.0000  18.0000
1745  Default   4.000000         network   4.000000  17.0000  17.0000
2387  Default   4.000000           spark   4.000000  16.0000  16.0000
2253  Default   9.000000      scientists   9.000000  15.0000  15.0000
534   Default   5.000000        concepts   5.000000  14.0000  14.0000
777   Default   8.000000         digital   8.000000  13.0000  13.0000
1791  Default   4.000000         october   4.000000  12.0000  12.0000
188   Default   4.000000           april   4.000000  11.0000  11.0000
1589  Default   4.000000           march   4.000000  10.0000  10.0000
2339  Default   6.000000          simple   6.000000   9.0000   9.0000
1417  Default   4.000000            july   4.000000   8.0000   8.0000
2292  Default   4.000000       september   4.000000   7.0000   7.0000
1403  Default   3.000000         january   3.000000   6.0000   6.0000
1028  Default   3.000000        february   3.000000   5.0000   5.0000
229   Default   3.000000          august   3.000000   4.0000   4.0000
1746  Default   7.000000        networks   7.000000   3.0000   3.0000
168   Default   4.000000          apache   4.000000   2.0000   2.0000
704   Default   3.000000        decision   3.000000   1.0000   1.0000
...       ...        ...             ...        ...      ...      ...
2777   Topic6  15.626317          weekly  23.407592   1.5092  -3.6660
1025   Topic6   1.463139         feature   2.203428   1.5039  -6.0344
1472   Topic6   0.967845            late   1.464284   1.4993  -6.4477
22     Topic6   1.711837              21   2.605011   1.4934  -5.8774
776    Topic6  15.670905          digest  24.894821   1.4505  -3.6632
2830   Topic6   0.836964           xpath   1.333357   1.4476  -6.5930
1850   Topic6   1.699724        overview   2.744452   1.4342  -5.8845
417    Topic6   0.798688  certifications   1.294574   1.4304  -6.6398
221    Topic6   0.788708         attacks   1.285270   1.4250  -6.6523
2339   Topic6   4.266740          simple   6.969962   1.4226  -4.9641
768    Topic6   0.783430      dictionary   1.280928   1.4217  -6.6591
1733   Topic6   1.437935         natural   2.357021   1.4191  -6.0518
1758   Topic6   2.238208             nlp   3.685222   1.4147  -5.6093
600    Topic6   1.331406     correlation   2.241601   1.3924  -6.1288
5      Topic6   1.740353              12   3.009695   1.3656  -5.8609
2258   Topic6   1.372653        scraping   2.374428   1.3653  -6.0982
2429   Topic6   3.686499     statistical   7.900105   1.1511  -5.1103
32     Topic6   1.723658              29   3.168204   1.3046  -5.8705
2006   Topic6   2.607956 